# Consolidación de la base de datos meteorológica con todas las fuentes disponibles 

## Metodología de la consolidación

Que vamos a hacer?
1. Descargar los 8 csv de AMSC, uno por cada atributo o varibles meteorológica
2. Consolidar en un mismo .csv, para que quede en una misma serie de tiempo |Fecha|Variables|
3. Reemuestreo de la BD a frecuencia por semana epidemiológica 
4. Preprocesamiento 

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Datos satélitales

Importación de datos de la NASA POWER

In [7]:
# Importar datos
nasa=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\NASA POWER\Datos_NS_2021-2024.csv", sep=';')
nasa

,YEAR,DOY,T2M,T2M_MAX,T2M_MIN,QV2M,RH2M,PRECTOTCORR,WS2M,WS2M_MAX,WS2M_MIN,WD2M
0,2021,1,28.38,34.96,23.58,16.62,71.66,3.59,0.14,0.24,0.02,355.4
1,2021,2,27.44,33.39,23.64,17.98,80.54,12.12,0.10,0.23,0.04,294.9
2,2021,3,28.64,35.09,23.84,17.89,75.41,4.03,0.14,0.32,0.07,332.0
3,2021,4,28.48,35.05,23.69,15.91,68.69,0.70,0.14,0.25,0.07,329.4
4,2021,5,27.84,34.96,22.71,15.16,68.47,0.62,0.16,0.34,0.03,310.1
...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2024,362,25.83,28.78,23.57,18.48,88.70,2.16,0.19,0.46,0.04,300.9
1457,2024,363,26.05,29.56,23.60,19.30,91.31,9.07,0.14,0.31,0.06,321.3
1458,2024,364,24.92,26.19,23.93,19.24,96.95,8.35,0.15,0.38,0.03,101.8
1459,2024,365,25.11,27.89,23.30,18.50,92.34,1.89,0.13,0.36,0.01,250.0


## Datos locales

Importación de los datos de AMSC


In [5]:
direccion_viento=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\AMSC_direccion_viento.csv", sep=';')
humedad=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\AMSC_humedad.csv", sep=';')
precipitacion=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\AMSC_precipitacion.csv", sep=';')
presion=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\AMSC_presion.csv", sep=';')
radiacion=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\AMSC_radiacion.csv", sep=';')
temperatura=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\AMSC_temperatura.csv", sep=';')
uv=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\AMSC_uv.csv", sep=';')
velocidad_viento=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\AMSC_velocidad_viento.csv", sep=';')

C:\Users\usuario1\AppData\Local\Temp\ipykernel_14328\3660575609.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  precipitacion=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\AMSC_precipitacion.csv", sep=';')


In [6]:
import pandas as pd
from functools import reduce

# Diccionario de tus dataframes y los nombres que quieres darles
dataframes = {
    "direccion_viento": direccion_viento,
    "humedad": humedad,
    "precipitacion": precipitacion,
    "presion": presion,
    "radiacion": radiacion,
    "temperatura": temperatura,
    "uv": uv,
    "velocidad_viento": velocidad_viento
}

# Renombrar columnas de forma segura
for name, df in dataframes.items():
    cols = df.columns.tolist()
    if len(cols) > 1:
        df.rename(columns={cols[1]: name}, inplace=True)
    else:
        print(f"⚠️ El archivo {name} solo tiene la columna: {cols}. No se pudo renombrar la variable.")

# Unir todos los dataframes por la columna 'fecha'
df_final = reduce(lambda left, right: pd.merge(left, right, on="fecha", how="outer"), dataframes.values())

# Convertir 'fecha' a tipo datetime
df_final["fecha"] = pd.to_datetime(df_final["fecha"], errors="coerce")

# Ordenar por fecha ascendente
df_final = df_final.sort_values(by="fecha", ascending=True).reset_index(drop=True)


In [7]:

# Mostrar primeras filas
df_final.iloc[20400:20420, :]



,fecha,direccion_viento,humedad,precipitacion,presion,radiacion,temperatura,uv,velocidad_viento
20400,2022-06-12 08:58:00,30.0,78.0,0.0,101.236.099.155,141.0,27.722.222.222.222.200,255.0,1.0
20401,2022-06-12 08:59:00,126.0,79.0,0.0,101.232.712.766,134.0,27.666.666.666.666.600,255.0,1.0
20402,2022-06-12 09:00:00,96.0,78.0,0.0,101.232.712.766,134.0,27.666.666.666.666.600,255.0,1.0
20403,2022-06-12 09:01:00,101.0,78.0,0.0,101.225.939.988,134.0,27.666.666.666.666.600,255.0,2.0
20404,2022-06-12 09:02:00,101.0,78.0,0.0,101.232.712.766,135.0,2.755.555.555.555.550,255.0,3.0
20405,2022-06-12 09:03:00,106.0,78.0,0.0,101.236.099.155,141.0,27.5,255.0,1.0
20406,2022-06-12 09:04:00,106.0,78.0,0.0,101.232.712.766,146.0,27.5,255.0,0.0
20407,2022-06-12 09:05:00,289.0,78.0,0.0,101.236.099.155,153.0,2.744.444.444.444.440,255.0,1.0
20408,2022-06-12 09:06:00,319.0,79.0,0.0,101.232.712.766,163.0,2.744.444.444.444.440,255.0,0.0
20409,2022-06-12 09:07:00,325.0,79.0,0.0,101.236.099.155,172.0,2.738.888.888.888.880,255.0,1.0


In [8]:
# Eliminar filas con al menos un dato faltante (NaN)
df_final_sin_na = df_final.dropna()

# Revisar resultado
df_final_sin_na.head(10)


,fecha,direccion_viento,humedad,precipitacion,presion,radiacion,temperatura,uv,velocidad_viento
1440,2022-01-12 00:00:00,257.0,94.0,0.0,100.992.279.147,0.0,24.277.777.777.777.700,255.0,1.0
1441,2022-01-12 00:01:00,327.0,94.0,0.0,100.985.506.369,0.0,2.433.333.333.333.330,255.0,0.0
1442,2022-01-12 00:02:00,327.0,94.0,0.0,100.985.506.369,0.0,2.433.333.333.333.330,255.0,0.0
1443,2022-01-12 00:03:00,327.0,94.0,0.0,100.978.733.591,0.0,2.433.333.333.333.330,255.0,0.0
1444,2022-01-12 00:04:00,360.0,94.0,0.0,100.971.960.813,0.0,2.433.333.333.333.330,255.0,1.0
1445,2022-01-12 00:05:00,360.0,94.0,0.0,100.978.733.591,0.0,2.433.333.333.333.330,255.0,1.0
1446,2022-01-12 00:06:00,40.0,94.0,0.0,100.978.733.591,0.0,2.433.333.333.333.330,255.0,1.0
1447,2022-01-12 00:07:00,59.0,94.0,0.0,10.098.211.998,0.0,2.433.333.333.333.330,255.0,0.0
1448,2022-01-12 00:08:00,60.0,94.0,0.0,10.098.211.998,0.0,2.433.333.333.333.330,255.0,1.0
1449,2022-01-12 00:09:00,81.0,94.0,0.0,100.985.506.369,0.0,2.433.333.333.333.330,255.0,0.0


In [9]:
#guardar df final csv
df_final_sin_na.to_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\Datos_meteorologicos_completos_2021-2024.csv", index=False)

Empezamos a hacer el reemuestreo 